In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import preprocess_input
from keras.utils import to_categorical

In [3]:
data_dir = '/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images'
csv_path = '/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train.csv'
images_dir = '/kaggle/working/images_dir'
labels = []

In [ ]:
import shutil

shutil.rmtree('/kaggle/working/images')
shutil.rmtree('/kaggle/working/labels')


In [ ]:
shutil.rmtree('/kaggle/working/training')

In [ ]:
os.makedirs('images', exist_ok=True)
os.makedirs('labels', exist_ok=True)

In [ ]:

train_df = pd.read_csv(csv_path)
labels=[]
images = np.empty((0, 224, 224))

In [ ]:

images = np.empty((0, 227, 227))
os.makedirs('training', exist_ok=True)

counter=0

for subdir in os.listdir(data_dir):
    if(counter < 20):
        counter+=1
        print(counter)
       
        if os.path.isdir(os.path.join(data_dir, subdir)):
            print('running')
            
            study_uid = subdir
            label = train_df[train_df['StudyInstanceUID'] == study_uid]['patient_overall'].values[0]
            
            for file in os.listdir(os.path.join(data_dir, subdir)):
                
                if file.endswith('.dcm'):
                    
                    try:
                        dcm_file = pydicom.read_file(os.path.join(data_dir, subdir, file))
                        dcm_file.PhotometricInterpretation = 'YBR_FULL'
                       
                        img = dcm_file.pixel_array.astype(float)
                        img = (img / np.max(img)) * 255.0
                   
                        img = Image.fromarray(img.astype('uint8'))
                        img = img.resize((227,227))
                        img.save(os.path.join('training', str(file) + '.jpg'))
                        img_arr = np.array(img)
                        images = np.concatenate((images, [img_arr]))
                        labels.append(label)
                    
                    except:
                        print("Skipping file:", file)
                        continue
                    

In [ ]:

!pip install gdcm

!pip install pylibjpeg

In [ ]:
!sudo apt-get install libgdcm2-dev

In [ ]:
from tensorflow.keras.utils import to_categorical
images=np.array(images)
labels=to_categorical(np.array(labels))

In [ ]:
images.shape

In [ ]:
labels

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
print(f'x_train: {len(x_train)}')
print(f'y_train: {len(y_train)}')
print(f'x_test: {len(x_test)}')
print(f'y_test: {len(y_test)}')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

input_shape = (227, 227, 3)

model = Sequential()

model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(units=4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
import numpy as np

x_train = np.expand_dims(x_train, axis=-1)

x_train = np.repeat(x_train, 3, axis=-1)


In [ ]:

x_test = np.expand_dims(x_test, axis=-1)

x_test = np.repeat(x_test, 3, axis=-1)

In [ ]:
x_train.shape

In [ ]:
y_train

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
model.fit(x_train, y_train, epochs=3, batch_size=32)


In [ ]:

test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt
import time

start_time = time.time()

history = model.fit(x_train, y_train, epochs=3)

end_time = time.time()

print("Total training time:", end_time - start_time, "seconds")


In [ ]:
start_time = time.time()

loss, test_accuracy = model.evaluate(x_test, y_test)

end_time = time.time()

print("Total testing time:", end_time - start_time, "seconds")


In [ ]:


plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('Accuracy vs. Loss')
plt.ylabel('Percentage')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Validation'], loc='upper left')
plt.show()